# <center> 3) Preparación de los Datos <center>

## Selección de Datos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
# !(pip install chainladder)
import chainladder as cl

print("pandas: " + pd.__version__)
print("numpy: " + np.__version__)
print("chainladder: " + cl.__version__)

pandas: 1.4.2
numpy: 1.22.0
chainladder: 0.8.18


In [2]:
data=pd.read_csv('wkcomp_pos.csv')

In [17]:
data
# data.columns[4:11]

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
0,86,Allstate Ins Co Grp,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872
1,86,Allstate Ins Co Grp,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872
2,86,Allstate Ins Co Grp,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872
3,86,Allstate Ins Co Grp,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872
4,86,Allstate Ins Co Grp,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13195,44300,Tower Ins Co Of NY,1997,2002,6,334,287,15,4387,3374,1013,1,1187
13196,44300,Tower Ins Co Of NY,1997,2003,7,318,293,2,4387,3374,1013,1,1187
13197,44300,Tower Ins Co Of NY,1997,2004,8,323,300,7,4387,3374,1013,1,1187
13198,44300,Tower Ins Co Of NY,1997,2005,9,310,297,0,4387,3374,1013,1,1187


### Triángulo de pérdidas

Creamos el triángulo de pérdidas para mirar como han evolucionado los pagos de las pólizas a partir del año en que ocurre el accidente, se toma como origen el año del accidente y como development el año en que se evalua el pago de la póliza. Se crean triángulos para cada una de las variables tanto de pérdidas (pólizas pagadas) como de ganancias (polizas ganadas en las compañías). El proceso se puede clasificar para cada compañía, por lo que se escoge como índice el nombre de las compañías GRNAME

In [41]:
triangle = cl.Triangle(
    data,
    origin="AccidentYear",
    development="DevelopmentYear",
    columns=data.columns[5:],
    index=["GRNAME"],
    cumulative=True,
)
triangle

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 1, 19, 19)"
Index:,[GRNAME]
Columns:,[EarnedPremCeded_D]


In [20]:
triangle.index.head()

,GRNAME
0,Agway Ins Co
1,Alaska Nat Ins Co
2,Alaska Timber Ins Exchange
3,Allstate Ins Co Grp
4,American Contractors Ins Grp


In [21]:
print("Estructura del Chainladder triangle:", type(triangle.values))
print("Suma de los valores de la tabla de datos:", np.nansum(triangle.values))

Estructura del Chainladder triangle: <class 'numpy.ndarray'>
Suma de los valores de la tabla de datos: 768171998.0


In [22]:
triangle[[x for x in data.columns if("Loss" in x)]]

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 3, 19, 19)"
Index:,[GRNAME]
Columns:,"[IncurLoss_D, CumPaidLoss_D, BulkLoss_D]"


Frecuencias de la parte exploratoria para GRNAME, la cual nos mostraba los nombres de las 132 empresas en la base de datos.

In [23]:
freq=pd.DataFrame(data['GRNAME'].value_counts())
freq.index

Index(['Allstate Ins Co Grp', 'Bancinsure Inc', 'Harco Natl Ins Co',
       'Transportation Cas Ins Co', 'Beacon Mut Ins Co', 'Mapfre Reins Corp',
       'National American Ins Co', 'Midwest Family Mut Ins Co',
       'Associated Industries Ins Co', 'Lumbermens Underwriting Alliance',
       ...
       'Catholic Relief Ins Co Of Amer', 'Care West Ins Co', 'FFVA Mut Ins Co',
       'MO Employers Mut Ins Co', 'Martingale Natl Ins Co',
       'Hyundai Marine & Fire Ins Co Ltd', 'Agway Ins Co', 'Mada Ins Exchange',
       'Lumber Ins Cos', 'Tower Ins Co Of NY'],
      dtype='object', length=132)

Acontinuación se muestra el triángulo para de las pérdidas pagadas y gastos al final de cada año y de forma acumulada. El año de inicio es de 1988 y a lo máximo el año de evaluación en el que se ha pagado llega al año 1997, esto a pesar de que hay datos de hasta el 2006

In [24]:
triangle.loc[freq.index[0]].iloc[-1]['CumPaidLoss_D']

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,"70,571","155,905","220,744","251,595","274,156","287,676","298,499","304,873","321,808","325,322",,,,,,,,,
1989,"66,547","136,447","179,142","211,343","231,430","244,750","254,557","270,059","273,873","277,574",,,,,,,,,
1990,"52,233","133,370","178,444","204,442","222,193","232,940","253,337","256,788","261,166","263,000",,,,,,,,,
1991,"59,315","128,051","169,793","196,685","213,165","234,676","239,195","245,499","247,131","248,319",,,,,,,,,
1992,"39,991","89,873","114,117","133,003","154,362","159,496","164,013","166,212","167,397","168,844",,,,,,,,,
1993,"19,744","47,229","61,909","85,099","87,215","88,602","89,444","89,899","90,446","90,686",,,,,,,,,
1994,"20,379","46,773","88,636","91,077","92,583","93,346","93,897","94,165","94,558","94,730",,,,,,,,,
1995,"18,756","84,712","87,311","89,200","90,001","90,247","90,687","91,068","91,001","91,161",,,,,,,,,
1996,"42,609","44,916","46,981","47,899","48,583","49,109","49,442","49,073","49,161","49,255",,,,,,,,,
1997,691,"2,085","2,795","2,866","2,905","2,909","2,908","2,909","2,909","2,909",,,,,,,,,


In [52]:
# for x in data.columns[5:]:
#     display(triangle.loc[freq.index[10]].iloc[-1][x])

### Creamos dos nuevas variables en el triángulo de pérdidas.

"CaseincurLoss": Esta nueva variable se crea restando el valor de "BulkLoss_D" de "IncurLoss_D". Básicamente, calcula la diferencia entre las pérdidas incurridas y las pérdidas a gglobal  para cada punto de datos correspondiente en el conjunto de datos.

"PaidToInc": Esta nueva variable se crea dividiendo el valor de "CumPaidLoss_D" por "IncurLoss_D". Básicamente, calcula la proporción de pérdidas incurridas que se han pagado hasta un punto específico en el tiempo.

In [44]:
triangle["CaseIncurLoss"] = triangle["IncurLoss_D"] - triangle["BulkLoss_D"]
triangle["PaidToInc"] = triangle["CumPaidLoss_D"] / triangle["IncurLoss_D"]
triangle[["CaseIncurLoss", "PaidToInc"]]

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 2, 19, 19)"
Index:,[GRNAME]
Columns:,"[CaseIncurLoss, PaidToInc]"


In [26]:
triangle["CumPaidLoss_D"].sum() / triangle["EarnedPremNet_D"].sum()

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,0.1690,0.3776,0.5116,0.5892,0.6412,0.6701,0.6917,0.7078,0.7269,0.7343,,,,,,,,,
1989,0.1712,0.3805,0.5100,0.5927,0.6419,0.6733,0.6952,0.7183,0.7279,0.7368,,,,,,,,,
1990,0.1703,0.4028,0.5409,0.6217,0.6696,0.6995,0.7277,0.7417,0.7538,0.7613,,,,,,,,,
1991,0.1683,0.3844,0.5102,0.5858,0.6331,0.6691,0.6852,0.7008,0.7097,0.7164,,,,,,,,,
1992,0.1567,0.3569,0.4636,0.5357,0.5855,0.6069,0.6262,0.6394,0.6485,0.6567,,,,,,,,,
1993,0.1379,0.2995,0.3863,0.4484,0.4784,0.5010,0.5148,0.5244,0.5325,0.5386,,,,,,,,,
1994,0.1353,0.2892,0.3830,0.4296,0.4601,0.4791,0.4918,0.5020,0.5094,0.5168,,,,,,,,,
1995,0.1314,0.2937,0.3677,0.4157,0.4449,0.4625,0.4754,0.4834,0.4902,0.4970,,,,,,,,,
1996,0.1576,0.3041,0.3875,0.4366,0.4673,0.4886,0.5035,0.5122,0.5206,0.5264,,,,,,,,,
1997,0.1541,0.3310,0.4261,0.4831,0.5174,0.5398,0.5540,0.5657,0.5720,0.5786,,,,,,,,,


In [27]:
triangle.groupby("GRNAME").sum()

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 8, 19, 19)"
Index:,[GRNAME]
Columns:,"[IncurLoss_D, CumPaidLoss_D, BulkLoss_D, EarnedPremDIR_D, EarnedPremCeded_D, EarnedPremNet_D, CaseIncurLoss, PaidToInc]"


In [28]:
pd.DataFrame(np.unique(triangle["GRNAME"]))

,0
0,Agway Ins Co
1,Alaska Nat Ins Co
2,Alaska Timber Ins Exchange
3,Allstate Ins Co Grp
4,American Contractors Ins Grp
...,...
127,Wisconsin American Mut Ins Co
128,Wisconsin Cnty Mut Ins Corp
129,Workers Comp Exch
130,Yasuda Fire & Marine Ins Co Of Amer


In [29]:
triangle.sum(axis="index").sum(axis="segments")

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(1, 8, 19, 19)"
Index:,[GRNAME]
Columns:,"[IncurLoss_D, CumPaidLoss_D, BulkLoss_D, EarnedPremDIR_D, EarnedPremCeded_D, EarnedPremNet_D, CaseIncurLoss, PaidToInc]"


In [30]:
triangle.sum(axis=0).sum(axis=1)

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,"6,505,202","6,997,869","7,251,030","7,372,099","7,515,941","7,561,299","7,568,639","7,594,218","7,626,114","7,627,609",,,,,,,,,
1989,"6,993,457","7,484,945","7,702,620","7,928,530","8,019,304","8,050,753","8,084,617","8,121,028","8,141,987","8,134,594",,,,,,,,,
1990,"7,497,081","8,222,617","8,450,322","8,596,685","8,650,997","8,678,112","8,728,115","8,764,842","8,775,955","8,762,355",,,,,,,,,
1991,"8,307,641","8,883,702","9,128,666","9,234,402","9,288,538","9,360,472","9,398,552","9,424,778","9,414,444","9,406,372",,,,,,,,,
1992,"8,511,840","9,181,715","9,294,013","9,316,201","9,383,361","9,437,915","9,472,147","9,475,233","9,493,746","9,488,202",,,,,,,,,
1993,"9,280,862","9,671,522","9,745,785","9,724,976","9,784,144","9,817,475","9,809,138","9,781,034","9,780,237","9,770,262",,,,,,,,,
1994,"9,631,596","9,962,505","9,981,473","9,963,320","9,983,721","9,994,179","9,982,154","9,978,186","9,984,483","9,999,210",,,,,,,,,
1995,"9,607,223","9,916,159","9,952,892","9,865,499","9,889,690","9,871,081","9,880,913","9,884,362","9,913,839","9,943,801",,,,,,,,,
1996,"8,998,547","9,221,501","9,366,476","9,353,886","9,346,671","9,365,625","9,382,031","9,403,202","9,438,648","9,465,932",,,,,,,,,
1997,"8,271,104","8,668,302","8,855,529","8,865,094","8,899,171","8,955,651","8,983,482","9,024,955","9,065,311","9,082,461",,,,,,,,,
